# Statistical Shape Models

## First, load the data:

In [ ]:
import numpy as np

nodes_train = np.load('./data/nodes_train.npy')
shape_train = nodes_train.shape

## Principal component Analysis
Re-arrange to put all the coodinates in individual vectors, so that we will have 200 of them. The take out the means:

In [ ]:
X = np.reshape(nodes_train,[642*3,200],order='F')  # use Fortran-like index order for ease of reshaping
mX = np.mean(X,axis=1,keepdims=True)
X = X - mX
print(X.shape)

Now compute eigendecompose of the Gram matrix (instead of the covariance matrix when observation is limited for numerical stability and computational efficiency), such that dot(G, evs) = lambdas * evs:

In [ ]:
G = np.matmul(np.transpose(X),X) / (shape_train[2]-1)
lambdas, evs = np.linalg.eig(G)
lambdas = np.clip(lambdas,1e-6,None)  # to numerically avoid negative variances
print(lambdas.shape)
print(evs.shape)

Inspect the numerical values of the eigendecomposition results:

In [ ]:
print(np.matmul(G,evs))

In [ ]:
print(np.matmul(evs,np.diag(lambdas)))  # lambdas*evs

First normalise the eigenvectors to unit-length, the principal components PCs. Then, sort the PCs and eigenvectors evs in descending order per in lambdas:

In [ ]:
PCs = np.matmul(np.matmul(X,evs),np.diag(1/np.sqrt((shape_train[2]-1)*lambdas)))
idx_sorted = np.flip(np.argsort(lambdas),axis=0)
lambdas = lambdas[idx_sorted]
PCs = PCs[:,idx_sorted] 
print(PCs.shape)
print(lambdas)

## Test the SMM reconstruction B = PCs' * X  ->  X = PCs * B:

In [ ]:
nPCs = 5
B = np.matmul(np.transpose(PCs[:,0:nPCs]), X)  # projection
X_recon = np.matmul(PCs[:,0:nPCs], B)  # reconstruction
diff = X_recon-X
print(PCs[:,0:nPCs].shape)
print(np.sqrt(np.mean(np.square(diff))))  # in mm

## Save the SSM "model" now:

In [ ]:
np.savez('./data/my_smm.npz', mX, PCs, lambdas)

## DIY
Now, given a fixed nPCs, change the weights of PCs between [-3 * sqrt(lambdas), 3 * sqrt(lambdas)]. One should expect to see the "shape variance" summarised by the SMM,
e.g. <img src="./media/Deformation_smm.gif" width=75% />

footnote: the PCA can also be performed by sigular value decomposition, which is numerically superior, while iterative methods, e.g. expectation-maximisation, also exists for additional computational gain or on-line performace.